# Exemple complet

## Objectif 

Compléter l'exemple simple avec un jeu de données réel.

- réseau autoroutier : 'roads_GL2017_Council_FR'
- stations IRVE : 'csl_gireve202407.csv'

In [12]:
import os
import sys

new_path = r"D:\philippe\python ESstandard\geonetworkx"
#new_path = os.getcwd()[:-8]
sys.path.append(new_path)
from shapely import LineString, Point
import numpy as np
import geopandas as gpd
import pandas as pd
import geonetworkx as gnx 
import networkx as nx 

from geonetworkx import geom_to_crs, cast_id 

## Création des données du réseau routier

Un GeoDataFrame pour les troncons routier

In [4]:
rtet = gpd.read_file("./roads_GL2017_Council_FR/roads_GL2017_Council_FR.shp").to_crs(2154)
print(rtet.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          487 non-null    object        
 1   COUNTRY_CO  487 non-null    object        
 2   MEMBER_STA  487 non-null    object        
 3   DESCRIPTIO  486 non-null    object        
 4   CORE_NETWO  487 non-null    object        
 5   CORRIDORS   181 non-null    object        
 6   TYPE        356 non-null    object        
 7   INTROADNUM  478 non-null    object        
 8   NATIONALRO  440 non-null    object        
 9   ORDER_ALL   487 non-null    int32         
 10  GEO_LENGTH  487 non-null    float64       
 11  OBJECT_MOD  487 non-null    object        
 12  EDIT_DATE   487 non-null    datetime64[ms]
 13  GIS_STATUS  487 non-null    object        
 14  SHAPE_Leng  487 non-null    float64       
 15  geometry    487 non-null    geometry      
dtypes: datetime64[ms](

In [28]:
rtet["debuts"] = rtet["geometry"].apply(lambda ls: ls.boundary.geoms[0])
rtet["fins"] = rtet["geometry"].apply(lambda ls: ls.boundary.geoms[1])
rtet

,ID,COUNTRY_CO,MEMBER_STA,DESCRIPTIO,CORE_NETWO,CORRIDORS,TYPE,INTROADNUM,NATIONALRO,ORDER_ALL,GEO_LENGTH,OBJECT_MOD,EDIT_DATE,GIS_STATUS,SHAPE_Leng,geometry,debuts,fins
0,22500102,FR,1,Artigues-près-Bordeaux (J. N89/N230) <--> Bègl...,1,G,None,E5/E70,N230,1038,8323.0,ROADS,2021-03-02,Completed,11748.070484,"LINESTRING (423285.813 6424900.789, 423296.286...",POINT (423285.813 6424900.789),POINT (420753.888 6417875.713)
1,22500101,FR,1,Lormont (J. A10/A630/N230) <--> Artigues-près-...,1,G,None,E5,N230,1040,2139.0,ROADS,2021-03-02,Completed,3022.615090,"LINESTRING (423285.813 6424900.789, 423260.582...",POINT (423285.813 6424900.789),POINT (423072.795 6426859.103)
2,22500134,FR,1,Brie-Comte-Robert (J. N104/N19) <--> Pontault-...,1,GHL,None,E000,N104,888,8477.0,ROADS,2021-10-01,Completed,12863.199413,"LINESTRING (671499.291 6852729.189, 671495.995...",POINT (671499.291 6852729.189),POINT (670281.883 6844870.297)
3,22500103,FR,1,Villenave-d'Ornon (J. A630/A62) <--> Bègles (J...,1,G,None,E5/E70/E72,A630,1039,3981.0,ROADS,2021-03-02,Completed,5612.447227,"LINESTRING (418500.207 6415064.574, 418545.009...",POINT (418500.207 6415064.574),POINT (420753.888 6417875.713)
4,22500177,FR,1,Balbigny (J. N82/A89) <--> La Tour-de-Salvagny...,1,None,None,E000,A89,1150,49153.0,ROADS,2021-03-02,Completed,70516.679994,"LINESTRING (831771.712 6525853.392, 831684.709...",POINT (831771.712 6525853.392),POINT (790652.69 6527082.72)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,22500184,FR,1,Isneauville (J. A28/N28) <--> Louviers (J. A13...,0,None,None,E000,None,875,35095.0,ROADS,2021-09-28,New Construction,53902.854777,"LINESTRING (567469.512 6934852.319, 567644.665...",POINT (567469.512 6934852.319),POINT (566604.154 6907339.779)
483,151008084937001,FR,1,Orgeval (J. A14/A13) <--> Méry-sur-Oise (J. N1...,0,None,None,E000,A104,900,22743.0,ROADS,2022-09-06,New Construction,34643.931266,"LINESTRING (641139.885 6883831.887, 640504.96 ...",POINT (641139.885 6883831.887),POINT (626544.302 6869985.499)
484,210302122851481,FR,1,Boulogne-sur-Mer (J. A16/N42) <--> Calais (J. ...,0,None,Motorways,E402,A16,925,8645.0,ROADS,2021-03-02,Completed,13699.149010,"LINESTRING (615673.704 7092971.595, 615705.899...",POINT (615673.704 7092971.595),POINT (622997.601 7093843.781)
485,211015123914126,FR,1,Saint-Denis <--> La Possession,0,None,Motorways,None,None,0,26246.0,ROADS,2022-09-06,Completed,28132.231998,"LINESTRING (10166078.109 519900.605, 10166834....",POINT (10166078.109 519900.605),POINT (10191695.527 551314.959)


In [59]:
nodes = pd.concat([rtet['debuts'], rtet['fins']]).drop_duplicates()
#nodes = nodes.reset_index().set_index([0])
nodidx=pd.Series(nodes.index, index=nodes)
nodidx

POINT (423285.813 6424900.789)        0
POINT (671499.291 6852729.189)        2
POINT (418500.207 6415064.574)        3
POINT (831771.712 6525853.392)        4
POINT (835680.917 6527610.007)        6
                                   ... 
POINT (1176586.683 6109253.119)     459
POINT (848685.752 6964570.676)      461
POINT (1052249.093 6831880.95)      480
POINT (10191695.527 551314.959)     485
POINT (-6438752.504 4266622.333)    486
Length: 415, dtype: int64

In [60]:
rtets = rtet.join(nodidx, on="debuts")
rtets

ValueError: Other Series must have a name

In [33]:
nod_tar = nodes.rename(columns={"index":"target"})
nod_tar

,target
0,
POINT (423285.813 6424900.789),0
POINT (671499.291 6852729.189),2
POINT (418500.207 6415064.574),3
POINT (831771.712 6525853.392),4
POINT (835680.917 6527610.007),6
...,...
POINT (1176586.683 6109253.119),459
POINT (848685.752 6964570.676),461
POINT (1052249.093 6831880.95),480


In [37]:
nod_ta = rtet["fins"]
#nod_ta[Point(420753.888, 6417875.713)]
nod_ta

0        POINT (420753.888 6417875.713)
1        POINT (423072.795 6426859.103)
2        POINT (670281.883 6844870.297)
3        POINT (420753.888 6417875.713)
4          POINT (790652.69 6527082.72)
                     ...               
482      POINT (566604.154 6907339.779)
483      POINT (626544.302 6869985.499)
484      POINT (622997.601 7093843.781)
485     POINT (10191695.527 551314.959)
486    POINT (-6438752.504 4266622.333)
Name: fins, Length: 487, dtype: geometry

In [47]:
print(nod_ta[0])

POINT (420753.88752062415 6417875.712820648)


In [32]:


rtets = rtet.join(nodes.rename(columns={"index":"source"}), on="debuts")
rtets = rtet.join(nodes.rename(columns={"index":"target"}), on="fins", validate='m:1')

rtets

,ID,COUNTRY_CO,MEMBER_STA,DESCRIPTIO,CORE_NETWO,CORRIDORS,TYPE,INTROADNUM,NATIONALRO,ORDER_ALL,GEO_LENGTH,OBJECT_MOD,EDIT_DATE,GIS_STATUS,SHAPE_Leng,geometry,debuts,fins,target
0,22500102,FR,1,Artigues-près-Bordeaux (J. N89/N230) <--> Bègl...,1,G,None,E5/E70,N230,1038,8323.0,ROADS,2021-03-02,Completed,11748.070484,"LINESTRING (423285.813 6424900.789, 423296.286...",POINT (423285.813 6424900.789),POINT (420753.888 6417875.713),0
1,22500101,FR,1,Lormont (J. A10/A630/N230) <--> Artigues-près-...,1,G,None,E5,N230,1040,2139.0,ROADS,2021-03-02,Completed,3022.615090,"LINESTRING (423285.813 6424900.789, 423260.582...",POINT (423285.813 6424900.789),POINT (423072.795 6426859.103),272
2,22500134,FR,1,Brie-Comte-Robert (J. N104/N19) <--> Pontault-...,1,GHL,None,E000,N104,888,8477.0,ROADS,2021-10-01,Completed,12863.199413,"LINESTRING (671499.291 6852729.189, 671495.995...",POINT (671499.291 6852729.189),POINT (670281.883 6844870.297),270
3,22500103,FR,1,Villenave-d'Ornon (J. A630/A62) <--> Bègles (J...,1,G,None,E5/E70/E72,A630,1039,3981.0,ROADS,2021-03-02,Completed,5612.447227,"LINESTRING (418500.207 6415064.574, 418545.009...",POINT (418500.207 6415064.574),POINT (420753.888 6417875.713),0
4,22500177,FR,1,Balbigny (J. N82/A89) <--> La Tour-de-Salvagny...,1,None,None,E000,A89,1150,49153.0,ROADS,2021-03-02,Completed,70516.679994,"LINESTRING (831771.712 6525853.392, 831684.709...",POINT (831771.712 6525853.392),POINT (790652.69 6527082.72),152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,22500184,FR,1,Isneauville (J. A28/N28) <--> Louviers (J. A13...,0,None,None,E000,None,875,35095.0,ROADS,2021-09-28,New Construction,53902.854777,"LINESTRING (567469.512 6934852.319, 567644.665...",POINT (567469.512 6934852.319),POINT (566604.154 6907339.779),253
483,151008084937001,FR,1,Orgeval (J. A14/A13) <--> Méry-sur-Oise (J. N1...,0,None,None,E000,A104,900,22743.0,ROADS,2022-09-06,New Construction,34643.931266,"LINESTRING (641139.885 6883831.887, 640504.96 ...",POINT (641139.885 6883831.887),POINT (626544.302 6869985.499),46
484,210302122851481,FR,1,Boulogne-sur-Mer (J. A16/N42) <--> Calais (J. ...,0,None,Motorways,E402,A16,925,8645.0,ROADS,2021-03-02,Completed,13699.149010,"LINESTRING (615673.704 7092971.595, 615705.899...",POINT (615673.704 7092971.595),POINT (622997.601 7093843.781),36
485,211015123914126,FR,1,Saint-Denis <--> La Possession,0,None,Motorways,None,None,0,26246.0,ROADS,2022-09-06,Completed,28132.231998,"LINESTRING (10166078.109 519900.605, 10166834....",POINT (10166078.109 519900.605),POINT (10191695.527 551314.959),485
